In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('../data/train_new_feat.csv')
df.head()

,track id,chroma_sftf,rolloff,zero_crossing_rate,rmse,flux,contrast,flatness,sample_silence,mfcc_0,...,power,joyful_activation,tension,sadness,mood,liked,disliked,age,gender,mother tongue
0,1,0.168468,5243.018296,0.115553,0.021488,1.004327,27.502646,0.00545,0,-404.308594,...,0,0,0,1,3,0,0,30,0,English
1,1,0.168468,5243.018296,0.115553,0.021488,1.004327,27.502646,0.00545,0,-404.308594,...,0,0,0,1,1,1,0,33,0,Russian
2,1,0.168468,5243.018296,0.115553,0.021488,1.004327,27.502646,0.00545,0,-404.308594,...,0,0,0,1,4,0,1,51,0,English
3,1,0.168468,5243.018296,0.115553,0.021488,1.004327,27.502646,0.00545,0,-404.308594,...,0,0,0,0,3,0,0,21,1,English
4,1,0.168468,5243.018296,0.115553,0.021488,1.004327,27.502646,0.00545,0,-404.308594,...,0,0,0,1,3,0,0,60,1,English


In [3]:
df['genre']

0       0
1       0
2       0
3       0
4       0
       ..
8402    3
8403    3
8404    3
8405    3
8406    3
Name: genre, Length: 8407, dtype: int64

In [4]:
music_emotion = [ ' tenderness', ' calmness', ' power',' joyful_activation', ' tension', ' sadness']
music_genre = ['genre']
user_features = [' age', ' gender',' mood']
LABELS = [' liked', ' disliked']
FEATURES = music_emotion + music_genre + user_features

In [6]:
def process_genre(db):
    """This function decodes the genre feature
       parameters:
           db (DataFrame): the data to process
       returns:
           a string corresponding to a music genre 
    """
    if db==0:
        return 'classical'
    elif db==1:
        return 'rock'
    elif db==2:
        return 'electronic'
    else:
        return 'pop'
# process the genre
df['genre'] = df['genre'] .apply(lambda x: process_genre(x), 1)
df['genre']

0       classical
1       classical
2       classical
3       classical
4       classical
          ...    
8402          pop
8403          pop
8404          pop
8405          pop
8406          pop
Name: genre, Length: 8407, dtype: object

In [11]:
train = df[FEATURES + LABELS]
# Process Audio Features: Onehot encode genre
train = pd.get_dummies(train, columns=['genre'])
cols = train.columns

# Rename Columns 
new_cols = []
for c in cols:
    new_cols.append(str(c).replace('genre_', ''))
train.columns = new_cols

In [12]:
train.columns

Index([' tenderness', ' calmness', ' power', ' joyful_activation', ' tension',
       ' sadness', ' age', ' gender', ' mood', ' liked', ' disliked',
       'classical', 'electronic', 'pop', 'rock'],
      dtype='object')

In [13]:
# Drop useless observations 
train['sum'] = train[LABELS].sum(1)
train = train[train['sum']==1]
train.drop(['sum', " disliked"], 1, inplace=True)

In [14]:
music_emotion = [' tenderness',
 ' calmness',
 ' power',
 ' joyful_activation',
 ' tension',
 ' sadness']
music_genre = ['classical',
       'electronic', 'pop', 'rock']

# Process user features : age grouping

In [15]:
simulation = train[music_emotion + music_genre + user_features]
# Process User Feautres
def process_age(db):
    """ This functions groups the age of the user
        parameters:
            db (DataFrame): the data to process
        returns
            a string corresponding to an age group
    """
    if db <= 12:
        return "child"
    elif db > 12 and db <= 25:
        return "young"
    elif db >25 and db <=35:
        return "young_adult"
    elif db > 35 and db <=50:
        return "adult"
    elif db > 50:
        return "senior"
    
simulation[' age'] = simulation[" age"].apply(lambda x: process_age(x), 1)
simulation = pd.get_dummies(simulation, columns=[' age'])
simulation['ratings'] = train[' liked']
simulation.head()

c:\python37\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,tenderness,calmness,power,joyful_activation,tension,sadness,classical,electronic,pop,rock,gender,mood,age_adult,age_child,age_senior,age_young,age_young_adult,ratings
1,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1
2,0,1,0,0,0,1,1,0,0,0,0,4,0,0,1,0,0,0
5,1,1,0,0,0,0,1,0,0,0,0,4,0,0,0,1,0,1
7,0,0,0,0,0,1,1,0,0,0,1,4,0,0,0,1,0,0
9,0,1,0,0,0,1,1,0,0,0,0,4,0,0,0,0,1,1


In [16]:
# Get HRV
hrv = pd.read_csv('../data/wesad-classification-hrv-train-dataset.csv')
hrv.head()

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,...,MEAN_RR_MEAN_MEAN_REL_RR,SD2_LF,HR_LF,HR_HF,HF_VLF,subject id,condition,SSSQ class,SSSQ Label,condition label
0,860.758450,858.389130,75.161448,14.461513,14.460581,5.197343,70.242263,6.25,0.75,10.238012,...,-5.208054e+06,0.137408,0.091228,3.666502,0.009514,7,baseline,low,0,0
1,1044.449374,1036.796500,82.209648,22.516679,22.515843,3.651056,57.792396,28.50,2.25,15.941094,...,4.953282e+06,0.059868,0.030044,25.537614,0.001175,4,baseline,low,0,0
2,510.072193,503.341590,32.094249,8.549018,8.547740,3.754145,118.059988,3.50,0.00,6.051753,...,-1.685542e+06,0.127065,0.333489,3.031509,0.084238,13,stress,medium,1,2
3,930.296969,940.394165,59.089578,7.816167,7.814190,7.559917,64.769049,0.00,0.00,5.532404,...,5.520502e+06,0.505268,0.392480,11.410422,0.004625,5,baseline,high,2,0
4,692.067906,687.546285,47.584437,6.487698,6.487256,7.334564,87.096547,0.00,0.00,4.592942,...,6.091203e+06,0.248241,0.322039,19.549568,0.003409,5,stress,high,2,2


# Map the stress level to the song rating (HRV dataset & Music dataset)

In [17]:
FEATURES = ['MEAN_RR','SDRR','HR','RMSSD','pNN50','VLF','LF','HF','subject id']
LABEL = ['SSSQ Label']

In [19]:
# get hrv with low stress level
hrv1 = hrv[hrv["SSSQ class"]=="low"]

#get hrv with high stress level
hrv0 = hrv[hrv["SSSQ class"]=="high"]

In [20]:
# Merge ratings and feelings
def get_hrv(db):
    """
    This function maps the stress level to the song rating
    
    parameters:
        db (DataFrame): the data to process
    returns:
        db (DataFrame): the processed data
    """
    global hrv1, hrv0
    # map good rating to low stress level
    if db['ratings'] == 1:
        sample = hrv1.sample(1).iloc[0]
        for f in FEATURES:
            db[f] = sample[f]
    # map bad ratings to high stress level
    else:
        sample = hrv0.sample(1).iloc[0]
        for f in FEATURES:
            db[f] = sample[f]
    return db     


In [21]:
%%time 
simulation = simulation.apply(get_hrv, 1)
simulation.head()

Wall time: 31.7 s


,tenderness,calmness,power,joyful_activation,tension,sadness,classical,electronic,pop,rock,...,ratings,MEAN_RR,SDRR,HR,RMSSD,pNN50,VLF,LF,HF,subject id
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,955.393597,145.513407,64.661410,22.833293,1.25,2727.188512,2100.772584,6.942776,4.0
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,495.137333,20.541448,121.380129,4.649531,0.00,122.992404,172.812347,9.815828,11.0
5,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1053.412598,89.129914,57.364457,26.631007,4.75,2025.397417,2910.018052,2.651670,4.0
7,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,684.916597,55.148493,88.146335,5.229125,0.00,729.405685,121.157088,4.408449,5.0
9,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,861.173850,75.949986,70.217896,15.026389,0.00,1795.627285,702.762118,32.335595,16.0


In [22]:
simulation = simulation.drop('subject id', 1)

In [23]:
simulation.columns

Index([' tenderness', ' calmness', ' power', ' joyful_activation', ' tension',
       ' sadness', 'classical', 'electronic', 'pop', 'rock', ' gender',
       ' mood', ' age_adult', ' age_child', ' age_senior', ' age_young',
       ' age_young_adult', 'ratings', 'MEAN_RR', 'SDRR', 'HR', 'RMSSD',
       'pNN50', 'VLF', 'LF', 'HF'],
      dtype='object')

# Save the dataset

In [25]:
simulation.to_csv('../data/simulation_training_hrv_recommender.csv',index=False) 